In [5]:
!pip install pyspark
!pip install sparknlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 692.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.5/579.5 kB 3.4 MB/s eta 0:00:00a 0:00:01


In [6]:
#For NLP Session
import sparknlp
spark = sparknlp.start()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5ea6d11d-da6d-401f-83d5-9c43c537bb43;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.4.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.g

In [10]:
from pyspark.sql.functions import lit

file_path_joe = '/kaggle/input/sentiment-analysis/hashtag_joebiden.csv'
file_path_trump = '/kaggle/input/sentiment-analysis/hashtag_donaldtrump.csv'
df_joe = spark.read.csv(file_path_joe, header=True, inferSchema=True)
df_trump = spark.read.csv(file_path_trump, header=True, inferSchema=True)

In [11]:
df_joe = df_joe.withColumn('candidate', lit('joe_biden'))
df_trump = df_trump.withColumn('candidate', lit('donald_trump'))

#Combining JoeBiden and Donald Trump Datasets
df=df_joe.union(df_trump)

In [12]:
df = df.filter(df['tweet'].isNotNull())
df = df.filter(df['state'].isNotNull())
df = df.distinct()
df.count()

844832

In [15]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType
from sparknlp.annotator import SentenceDetector


In [16]:
MODEL_NAME='sentimentdl_use_twitter'

documentAssembler = DocumentAssembler()\
    .setInputCol("tweet")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

languageDetector = LanguageDetectorDL.pretrained()\
            .setInputCols("document")\
            .setOutputCol("language")\
            .setThreshold(0.5)\
            .setCoalesceSentences(True)

sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment_view")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler,
        use,
        languageDetector,
        sentimentdl
        ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]

24/09/09 00:17:36 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/09/09 00:17:36 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]Download done! Loading the resource.
[ / ]

[OK!]
ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[ | ]

24/09/09 00:18:54 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/09/09 00:18:54 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB


24/09/09 00:18:54 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


[ / ]Download done! Loading the resource.
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ]

24/09/09 00:18:59 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/09/09 00:18:59 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/09/09 00:18:59 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
Download done! Loading the resource.
[OK!]


In [20]:
result = nlpPipeline.fit(df).transform(df)

df_new=result.select('*',F.explode(F.arrays_zip(result.language.result,result.sentiment_view.result)).alias("cols"))\
        .select('*',F.expr("cols['0']").alias("languages_det"),
                F.expr("cols['1']").alias("sentiment"))
        
        
from pyspark.sql.functions import rand

df_new = df_new.orderBy(rand()) #shuffling rows show that equal proportion of rows can be divided during training and testing phase
df_new.cache()

DataFrame[created_at: string, tweet_id: string, tweet: string, likes: string, retweet_count: string, source: string, user_id: string, user_name: string, user_screen_name: string, user_description: string, user_join_date: string, user_followers_count: string, user_location: string, lat: string, long: string, city: string, country: string, continent: string, state: string, state_code: string, collected_at: string, candidate: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence_embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, language: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentiment_view: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, cols: struct<0:string,1:

In [36]:
#Designing of Model

In [21]:
#Selecting english tweet
a=df_new.coalesce(1)
df_new=a.filter(df_new.languages_det=='en')
df_new=df_new.filter(df_new.country=='United States of America')
df_new=df_new.select('tweet','candidate')
df_new = df_new.filter(df_new.tweet.isNotNull())
df_new = df_new.filter(df_new.candidate.isNotNull())

In [22]:
#Splitting the data between training and test set

seed = 1231768
total_train, total_test = df_new.randomSplit([0.7,0.3],seed)
total_train.cache()
total_test.cache()

DataFrame[tweet: string, candidate: string]

In [23]:
#Creat label for the train set
joe_biden = total_train.filter(total_train.candidate=='joe_biden').count()
donald_trump = total_train.filter(total_train.candidate=='donald_trump').count()

In [40]:
print(f'joe_biden vs donald_trump {joe_biden}, {donald_trump}, {joe_biden/donald_trump}')
print(f'train and test count: {total_train.count()}, {total_test.count()}')

joe_biden vs donald_trump 49852, 61657, 0.8085375545355759


train and test count: 111509, 47867


In [24]:
labeled=total_train.sample(False,0.01,seed)
#labeled.count()

In [25]:
#For manually labeling the sentiments based on tweets

joe_biden_labeled = labeled.filter(labeled.candidate=='joe_biden').count()
donald_trump_labeled = labeled.filter(labeled.candidate=='donald_trump').count()

print(f'joe_biden vs donald_trump {joe_biden_labeled}, {donald_trump_labeled}, {joe_biden_labeled/donald_trump_labeled}')
print(f'train and test count: {total_train.count()}, {total_test.count()}')

joe_biden vs donald_trump 512, 649, 0.7889060092449923


train and test count: 111509, 47867


In [43]:
# labeled_with_index = labeled_1.withColumn("row_index", monotonically_increasing_id())
# start_row = 0
# end_row = 1187

# # Select specific rows (100 to 240) and columns "candidate", "tweet", and "row_index"
# selected_rows_and_columns = labeled_with_index.filter((col("row_index") >= start_row) & (col("row_index") <= end_row)) \
#                                                .select("candidate", "tweet", "row_index")

# # Drop the "row_index" column 


In [45]:
#Manual Labelled Data
file_path_0 = '/kaggle/input/sentiment-analysis/0_manual_label.csv'
file_path_1 = '/kaggle/input/sentiment-analysis/1_manual_label.csv'
file_path_2 = '/kaggle/input/sentiment-analysis/2_manual_label.csv'
file_path_3 = '/kaggle/input/sentiment-analysis/3_manual_label.csv'
file_path_4 = '/kaggle/input/sentiment-analysis/4_manual_label.csv'


df_0 = spark.read.csv(file_path_0, header=True, inferSchema=True)
df_1 = spark.read.csv(file_path_1, header=True, inferSchema=True)
df_2 = spark.read.csv(file_path_2, header=True, inferSchema=True)
df_3 = spark.read.csv(file_path_3, header=True, inferSchema=True)
df_4 = spark.read.csv(file_path_4, header=True, inferSchema=True)


In [46]:
df_combine=df_amit.union(df_fanzel).union(df_evan).union(df_guneek).union(df_tingjian)
#df_combine.show(1)

In [47]:
from pyspark.sql.functions import col

df_combine=df_combine.drop('row_index')
#df_combine = df_combine.withColumnRenamed('tweet','new_tweet')
df_combine.columns
df_combine = df_combine.filter(df_combine['tweet'].isNotNull())
df_combine = df_combine.filter(df_combine['label'].isNotNull())
valid_values = ["positive", "negative", "neutral"]
df_combine = df_combine.filter(col('label').isin(valid_values))

b=df_combine.coalesce(1)


In [48]:
#Training and testing on label data

seed = 1231768
train, test = b.randomSplit([0.7,0.3],seed)
train.cache()
test.cache()

DataFrame[candidate: string, tweet: string, label: string]

In [49]:
labeled=train.sample(False,0.1,seed)
#labeled1.count()
unlabeled= train.subtract(labeled)

In [50]:
#Decision Tree

In [52]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetector
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.classification import DecisionTreeClassifier
import timeit
# Define DocumentAssembler stage
document_assembler = DocumentAssembler() \
    .setInputCol("tweet") \
    .setOutputCol("document")

# Define the Sentence Detector Stage
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
    
tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")
    
# Define Finisher
finisher = Finisher() \
    .setInputCols(['token']) \
    .setOutputCols(['result']) \
    .setOutputAsArray(True)
    
# Define CountVectorizer
count_vectorizer = CountVectorizer() \
    .setInputCol('result') \
    .setOutputCol('bow')

label_indexer = StringIndexer(inputCol="label", outputCol="label_index")

dt_classifier = DecisionTreeClassifier(labelCol='label_index', featuresCol='bow', seed=32)

# Create text processing pipeline
text_pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,tokenizer,
    finisher,count_vectorizer,label_indexer, dt_classifier
])


# Fit and transform the text processing pipeline to your data
start = timeit.default_timer() #scalability matrix

text_pipeline_model = text_pipeline.fit(labeled)
text_result_unlabeled = text_pipeline_model.transform(unlabeled)
text_set = text_pipeline_model.transform(test)

stop = timeit.default_timer()
print('Time' + str(stop - start))

Time5.880662796999786


In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Predict on test data
predictions_labeled_test_set = text_set
predictions_unlabeled = text_result_unlabeled

# Evaluate predictions using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy_predictions_labeled_test_set = evaluator.evaluate(predictions_labeled_test_set)
accuracy_predictions_unlabeled = evaluator.evaluate(predictions_unlabeled)


print("Accuracy accuracy_predictions_labeled_test_set:", accuracy_predictions_labeled_test_set)
print("Accuracy predictions_unlabeled:", accuracy_predictions_unlabeled)

# # Optionally, you can compute other evaluation metrics
# precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
# recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
# f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
# confusion_matrix = predictions.groupBy("label_index", "prediction").count()
# confusion_matrix.show()


Accuracy accuracy_predictions_labeled_test_set: 0.4186046511627907
Accuracy predictions_unlabeled: 0.3684210526315789


In [54]:
#Random Forest

In [55]:
from pyspark.ml.classification import RandomForestClassifier

rf_classifier = RandomForestClassifier(labelCol="label_index", featuresCol="bow",seed=32)

# Create text processing pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,tokenizer,
    finisher,count_vectorizer,label_indexer, rf_classifier
])

# Fit and transform the text processing pipeline to your data

# Fit the pipeline to the labeled data
model_labeled = pipeline.fit(labeled)
model_test = pipeline.fit(test)

# Predict on the labeled and test data
predictions_unlabeled_model = model_labeled.transform(unlabeled)
predictions_test_model = model_test.transform(test)

# Evaluate the model using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy_predictions_unlabeled_model = evaluator.evaluate(predictions_unlabeled_model)
accuracy_predictions_test_model = evaluator.evaluate(predictions_test_model)

print("Accuracy predictions_unlabeled:", accuracy_predictions_unlabeled_model)
print("Accuracy predictions_test_model:", accuracy_predictions_test_model)


Accuracy predictions_unlabeled: 0.39971550497866287
Accuracy predictions_test_model: 0.5


In [56]:
#Now testing it on the entire label training using Random Forest

ent_train=pipeline.fit(train)
ent_test=ent_train.transform(test)

predictions_test_label = ent_test

evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_test_label)
print("Accuracy on total label test set:", accuracy) 


Accuracy on total label test set: 0.4476744186046512


In [57]:
#Removing manual label rows from df_new data set for getting the labels
df_combine1=df_combine.select('candidate','tweet')
df_new=df_new.subtract(df_combine1)

In [58]:
# df_new1=df_new.limit(1000)
# df_new1=df_new1.coalesce(1)

In [59]:
text_set = text_pipeline_model.transform(df_new).select('candidate','tweet','prediction')
text_set.cache()
text_set=text_set.coalesce(1)
#text_set.show(5)

24/09/08 22:44:23 WARN StringIndexerModel: Input column label does not exist during transformation. Skip StringIndexerModel for this column.


In [60]:
combined_data=b.union(text_set.withColumnRenamed('prediction','label'))
combined_data=combined_data.coalesce(1)

In [61]:
train1, test1 = combined_data.randomSplit([0.7,0.3],seed)
train1=train1.coalesce(1)
test1= test1.coalesce(1)

In [62]:
#training and testing on the entire test set using Random Forest
combined_pipeline_model = pipeline.fit(train1)
combined_result = combined_pipeline_model.transform(test1)

24/09/08 22:50:11 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
24/09/08 22:50:12 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
24/09/08 22:52:34 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
24/09/08 23:00:31 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB
24/09/08 23:01:41 WARN MemoryStore: Not enough space to cache rdd_897_0 in memory! (computed 7.9 GiB so far)
24/09/08 23:01:41 WARN BlockManager: Persisting block rdd_897_0 to disk instead.
24/09/08 23:09:27 WARN MemoryStore: Not enough space to cache rdd_897_0 in memory! (computed 7.9 GiB so far)
24/09/08 23:10:45 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB
24/09/08 23:10:52 WARN MemoryStore: Not enough space to cache rdd_897_0 in memory! (computed 7.9 GiB so far)
24/09/08 23:12:12 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
24/09/08 23:12:19 WARN MemoryStore: Not enough space to cache rdd_897_0 in m

In [63]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(combined_result)
print("Accuracy on total set:", accuracy)

24/09/08 23:16:30 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


Accuracy on total set: 0.608036311914324
